In [23]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_chroma import Chroma
from dotenv import load_dotenv
import os

In [21]:
loader = DirectoryLoader("/Users/hritik/Desktop/Grading Doc RAG/Document", glob="**/*.html",loader_cls=UnstructuredHTMLLoader)
docs = loader.load()
len(docs)

166

In [24]:
dotenv_path = '../.env'
load_dotenv(dotenv_path)
HUGGINGFACEHUB_API_TOKEN = os.getenv("HF_TOKEN")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN
embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key=HUGGINGFACEHUB_API_TOKEN, model_name="sentence-transformers/all-MiniLM-l6-v2"
)

In [25]:
db = Chroma.from_documents(docs, embeddings, persist_directory='vectordb/chroma/')

In [28]:
print(db._collection.count())

200


In [29]:
ques = "Give me information about scholarship"

In [30]:
docs = db.similarity_search(ques,k=3)

In [31]:
docs

[Document(page_content='English\n\n| हिंदी\n\n| தமிழ்\n\n| తెలుగు\n\n| മലയാളം\n\n| ಕನ್ನಡ\n\n| मराठी\n\n| ગુજરાતી\n\n| বাংলা\n\nAcademics\r\n                            \xa0\r\n                            \r\n                        \r\n                        \r\n                            Overall Structure\r\n                            Term Structure\r\n                            Course Registrations\r\n                            Assessments\r\n                            \r\n                            Exam Cities\r\n                            \r\n                            Fee Structure\r\n                            \r\n                            Foundational Level Courses\r\n                            Diploma Level Courses\r\n                            Degree Level Courses\r\n                            \r\n                            Sample Certificates\n\nAdmissions\r\n                            \r\n                        \r\n                        \r\n              